In [9]:
import pickle
import pandas as pd
import os

In [10]:
import pickle
import pandas as pd
import os

In [11]:
with open('config_list13', 'rb') as f:
    config_list = pickle.load(f)
non_para = 0
exist_para = 0

datasets = ['chameleon', 'cornell', 'squirrel', 'film',
            'texas', 'wisconsin', 'pubmed', 'cora', 'citeseer']
config_str = ['lr', 'do', 'es', 'wd', 'alpha', 'beta',
              'gamma', 'delta', 'nlid', 'nl', 'ordersid', 'orders']

num_map = [0, 1, 2, 3, 4, 5, 6, 9, 10, 7, 11, 8, 12, 13]

In [12]:
config_list

[[0.01, 0.0, 200, 0.0, 0.0, 100000000.0, 0.0, 2, 1, 0.0, 1, 2, 0],
 [0.01, 0.0, 200, 0.0, 0.0, 100000000.0, 0.0, 2, 1, 0.1, 1, 2, 0],
 [0.01, 0.0, 200, 0.0, 0.0, 100000000.0, 0.0, 2, 1, 0.2, 1, 2, 0],
 [0.01, 0.0, 200, 0.0, 0.0, 100000000.0, 0.0, 2, 1, 0.3, 1, 2, 0],
 [0.01, 0.0, 200, 0.0, 0.0, 100000000.0, 0.0, 2, 1, 0.4, 1, 2, 0],
 [0.01, 0.0, 200, 0.0, 0.0, 100000000.0, 0.0, 2, 1, 0.5, 1, 2, 0],
 [0.01, 0.0, 200, 0.0, 0.0, 100000000.0, 0.0, 2, 1, 0.6, 1, 2, 0],
 [0.01, 0.0, 200, 0.0, 0.0, 100000000.0, 0.0, 2, 1, 0.7, 1, 2, 0],
 [0.01, 0.0, 200, 0.0, 0.0, 100000000.0, 0.0, 2, 1, 0.8, 1, 2, 0],
 [0.01, 0.0, 200, 0.0, 0.0, 100000000.0, 0.0, 2, 1, 0.9, 1, 2, 0],
 [0.01, 0.0, 200, 0.0, 0.0, 100000000.0, 0.0, 2, 1, 1.0, 1, 2, 0],
 [0.01, 0.0, 200, 5e-05, 0.0, 1.0, 0.0, 2, 1, 0.0, 1, 2, 1],
 [0.01, 0.0, 200, 5e-05, 0.0, 1.0, 0.0, 2, 1, 0.1, 1, 2, 1],
 [0.01, 0.0, 200, 5e-05, 0.0, 1.0, 0.0, 2, 1, 0.2, 1, 2, 1],
 [0.01, 0.0, 200, 5e-05, 0.0, 1.0, 0.0, 2, 1, 0.3, 1, 2, 1],
 [0.01, 0.0, 200, 5

In [13]:
print('the number of all parameters', len(config_list))
results_all = [[] for _ in range(len(config_str) + 2)]
for config in config_list:
    acc_res = []
    for i in range(10):
        url_configs = ''
        for j in range(len(config_str)):
            url_configs += '_' + config_str[j] + str(config[num_map[j]])
        url = 'runs/' + datasets[config[-1]] + url_configs + \
            '_split' + str(i) + '_results.txt'
        # print(url)
        if os.path.exists(url):
            # print('url exists')
            with open(url, 'r') as f:
                s = f.read()
                sub1 = '"test_acc": '
                sub2 = ', "test_duration"'
                result = s[s.index(sub1) + len(sub1): s.index(sub2)]
                acc_res.append(float(result))
    # print(url)
    # print(len(acc_res))
    if len(acc_res) == 10:
        exist_para += 1
        results_all[0].append(datasets[config[-1]])
        for i in range(len(config_str)):
            results_all[i+1].append(config[num_map[i]])
        results_all[-1].append(sum(acc_res)/len(acc_res))
    else:
        non_para += 1
        print(url)

print('the number of non-existing parameters', non_para)
print('the number of existing parameters', exist_para)

the number of all parameters 99
the number of non-existing parameters 0
the number of existing parameters 99


In [14]:
results_all[7]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8,
 0.8]

In [15]:
all_config_str = ['dataset', 'lr', 'dropout', 'early_stopping', 'weight_decay',
                  'alpha', 'beta', 'gamma', 'delta', 'norm_func_id', 'norm_layers',
                  'orders_func_id', 'orders', 'accuracy']
d = {}
assert len(all_config_str) == len(results_all)
for i in range(len(all_config_str)):
    d[all_config_str[i]] = results_all[i]

df = pd.DataFrame.from_dict(d)
df = df.sort_values(['dataset', 'accuracy']).reset_index(drop=True)

csv_name = 'results/mlpnorm_result_1+2.csv'
if os.path.exists(csv_name):
    df.to_csv(csv_name, mode='w', header=True)
else:
    df.to_csv(csv_name)

In [16]:
df

,dataset,lr,dropout,early_stopping,weight_decay,alpha,beta,gamma,delta,norm_func_id,norm_layers,orders_func_id,orders,accuracy
0,chameleon,0.01,0.0,200,0.00000,0.0,100000000.0,0.0,0.0,1,2,2,1,0.654605
1,chameleon,0.01,0.0,200,0.00000,0.0,100000000.0,0.0,0.1,1,2,2,1,0.657456
2,chameleon,0.01,0.0,200,0.00000,0.0,100000000.0,0.0,0.2,1,2,2,1,0.659430
3,chameleon,0.01,0.0,200,0.00000,0.0,100000000.0,0.0,0.3,1,2,2,1,0.662939
4,chameleon,0.01,0.0,200,0.00000,0.0,100000000.0,0.0,0.4,1,2,2,1,0.667763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,wisconsin,0.01,0.0,200,0.00005,0.0,1.0,0.0,0.6,1,2,2,1,0.611765
95,wisconsin,0.01,0.0,200,0.00005,0.0,1.0,0.0,0.7,1,2,2,1,0.674510
96,wisconsin,0.01,0.0,200,0.00005,0.0,1.0,0.0,0.8,1,2,2,1,0.766667
97,wisconsin,0.01,0.0,200,0.00005,0.0,1.0,0.0,0.9,1,2,2,1,0.847059
